In [1]:
import numpy as np
np.random.seed(0)

In [2]:
def matmul(A, B):
    if A.shape[1] != B.shape[0]:
        print("Matrix dims are not compatible")
        return None
    m = A.shape[0]
    r = A.shape[1]
    n = B.shape[0]
    out = np.zeros((m, n), dtype=float) 
    for i in range(m):
        for j in range(n):
            for k in range(r):
                out[i,j] += A[i,k]*B[k,j]
    return out

In [3]:
A = np.random.normal(size=(1000, 200))
B = np.random.normal(size=(200, 1000))

In [4]:
%timeit -r 1 -n 1 matmul(A,B)

14.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
import numpy as np
from numba import jit, prange

@jit(nopython=True, parallel=True)
def matmul_numba(A, B):
    m = A.shape[0]
    r = A.shape[1]
    n = B.shape[0]
    out = np.zeros((m, n)) #numba works well with numpy
    for i in prange(m):
        for j in range(n):
            for k in range(r):
                out[i,j] += A[i,k]*B[k,j]
    return out

In [6]:
import time
start = time.time(); matmul_numba(A,B); print(f'Initial run: {time.time() - start} seconds')

Initial run: 0.8323376178741455 seconds


In [7]:
#subsequent run
%timeit matmul_numba(A,B)

5.74 ms ± 178 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%load_ext cython

In [ ]:
%%cython -a --compile-args=-fopenmp --link-args=-fopenmp --force
import numpy as np
import cython
cimport numpy as np
from cython.parallel import prange

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
def matmul_cython(np.ndarray[np.float64_t, ndim=2, mode='c'] A, np.ndarray[np.float64_t, ndim=2, mode='c'] B):
    #mode = 'c' gives that the matrices are stored in row-major order
    #this is numpy's default.
    if A.shape[1] != B.shape[0]:
        print("Matrix dims are not compatible")
        return None
    cdef: 
        int m = A.shape[0]
        int r = A.shape[1]
        int n = B.shape[0]
        int i, j, k
        np.ndarray[np.float64_t, ndim=2, mode='c'] out = np.zeros((m, n), dtype=float)
    
    #or you can wrap this in a "with nogil:" block
    for i in prange(m, nogil=True, num_threads=16):
        for k in range(r):
            for j in range(n): #using j as innermost loop improves speed 4x
                out[i,j] += A[i,k]*B[k,j]
    return out

In [10]:
%timeit matmul_cython(A,B) 
#cython is usually a little bit faster than numba
#here they're closer b/c the cython version has an extra if statement

5.74 ms ± 550 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
A = np.random.normal(size=(10000, 10000))
B = np.random.normal(size=(10000, 10000))

In [12]:
%timeit -r 1 -n 1 matmul_numba(A,B)

8min 32s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
%timeit -r 1 -n 1 matmul_cython(A,B)

2min 46s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [14]:
#back of the envelope calculations give that the 
#pure python version would take 
#80000 seconds = 22 hours for this final matmul

In [15]:
%timeit -r 1 -n 1 A @ B #wow! 
#Important to not reinvent the wheel when you don't have to

8.44 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [16]:
import math
def divisors(n):
    '''compute all divisors of n which are <= sqrt(n)'''
    divides_n = []
    for i in range(2, math.ceil(math.sqrt(n))):
        if n%i == 0:
            divides_n.append(i)
    return divides_n

In [17]:
from multiprocessing import Pool
P = Pool(10) #10 separate processes

In [ ]:
P.map(divisors, range(2, 1000000))

In [19]:
#parallel version (multiprocessing)
#compute all divisors <= sqrt(n) of all numbers
#n from 2 to 1 million
%timeit -r 1 -n 1 _ = P.map(divisors, range(2, 1000000))

5.95 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [20]:
#serial version
%timeit -r 1 -n 1 _ = list(map(divisors, range(2, 1000000)))

30.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
